ref: https://github.com/google/flax/tree/main/examples/mnist

In [90]:
# Copyright 2022 The Flax Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""MNIST example.

Library file which executes the training and evaluation loop for MNIST.
The data is loaded using tensorflow_datasets.
"""

# See issue #620.
# pytype: disable=wrong-keyword-args

from absl import logging
from flax import linen as nn
#from flax.metrics import tensorboard
from flax.training import train_state
import jax
import jax.numpy as jnp
#import ml_collections
import numpy as np
import optax
#import tensorflow_datasets as tfds


class CNN(nn.Module):
  """A simple CNN model."""

  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    return x


def create_train_state(rng, config):
  """Creates initial `TrainState`."""
  cnn = CNN()
  params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(config['learning_rate'], config['momentum'])
  return train_state.TrainState.create(apply_fn=cnn.apply, params=params, tx=tx)


@jax.jit
def apply_model(state, images, labels):
  """Computes gradients, loss and accuracy for a single batch."""
  def loss_fn(params):
    logits = state.apply_fn({'params': params}, images)
    one_hot = jax.nn.one_hot(labels, 10)
    loss = jnp.mean(optax.softmax_cross_entropy(logits=logits, labels=one_hot))
    return loss, logits

  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  print(state.params)
  (loss, logits), grads = grad_fn(state.params)
  accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
  return grads, loss, accuracy


@jax.jit
def update_model(state, grads):
  return state.apply_gradients(grads=grads)


def train_epoch(state, train_ds, y, batch_size, rng):
  """Train for a single epoch."""
  train_ds_size = len(train_ds)
  steps_per_epoch = train_ds_size // batch_size

  perms = jax.random.permutation(rng, len(train_ds))
  perms = perms[:steps_per_epoch * batch_size]  # skip incomplete batch
  perms = perms.reshape((steps_per_epoch, batch_size))
  print(perms.shape)

  epoch_loss = []
  epoch_accuracy = []

  for perm in perms:
    batch_images = train_ds[perm, ...].reshape(batch_size,28,28,1)  #train_ds['image'][perm, ...]
    batch_labels = y[perm, ...]  #train_ds['label'][perm, ...]
    grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
    state = update_model(state, grads)
    epoch_loss.append(loss)
    epoch_accuracy.append(accuracy)
  train_loss = np.mean(epoch_loss)
  train_accuracy = np.mean(epoch_accuracy)
  return state, train_loss, train_accuracy



In [3]:
from sklearn.datasets import fetch_openml

X,y = fetch_openml('mnist_784', return_X_y=True)


In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

seed = 1234

X_train, X_test, y_train, y_test = train_test_split(X / 255.0, y, test_size=0.2, random_state=seed)

In [25]:
config = {
  'learning_rate': 0.1,
  'num_epochs': 3,
  'batch_size': 128,
  'momentum': 0.9
}

rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)
print(config)
state = create_train_state(init_rng, config)


{'learning_rate': 0.1, 'num_epochs': 3, 'batch_size': 128, 'momentum': 0.9}


In [94]:

for epoch in range(1, config['num_epochs'] + 1):
  rng, input_rng = jax.random.split(rng)
  state, train_loss, train_accuracy = train_epoch(state, jnp.array(X_train), jnp.array(y_train, dtype=jnp.int16),
                                                  config['batch_size'],input_rng)
  _, test_loss, test_accuracy = apply_model(state, jnp.array(X_test).reshape(-1,28,28,1), jnp.array(y_test, dtype=jnp.int16))

  logging.info(
      'epoch:% 3d, train_loss: %.4f, train_accuracy: %.2f, test_loss: %.4f, test_accuracy: %.2f'
      % (epoch, train_loss, train_accuracy * 100, test_loss,
         test_accuracy * 100))

  print(f"epoch {epoch} -> train loss: {train_loss}, train_accuracy: {train_accuracy}, \
                           test loss: {test_loss}, test_accuracy: {test_accuracy}")


(437, 128)
FrozenDict({
    Conv_0: {
        bias: Traced<ShapedArray(float32[32])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[3,3,1,32])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Conv_1: {
        bias: Traced<ShapedArray(float32[64])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[3,3,32,64])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Dense_0: {
        bias: Traced<ShapedArray(float32[256])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[3136,256])>with<DynamicJaxprTrace(level=0/1)>,
    },
    Dense_1: {
        bias: Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(float32[256,10])>with<DynamicJaxprTrace(level=0/1)>,
    },
})
epoch 1 -> train loss: 0.01981988362967968, train_accuracy: 0.9936177134513855,                            test loss: 0.035975679755210876, test_accuracy: 0.9893571138381958
(437, 128)
epoch 2 -> tra